# Intro to Python ctypes
### Philipp Schindler

Slides available at:  
https://github.com/PhilippSchindler/ctypes-demo

See also:  
https://docs.python.org/3/library/ctypes.html

In [ ]:
from ctypes import cdll
from ctypes import c_char, c_int, c_uint32, c_double, sizeof, byref, POINTER

## Loading a shared library

In [ ]:
libc = cdll.LoadLibrary("libc.so.6")
libc

In [ ]:
import ctypes.util

name = ctypes.util.find_library('sodium')
libsodium = ctypes.cdll.LoadLibrary(name)
name, libsodium

In [ ]:
try:
    ctypes.cdll.LoadLibrary("~/test/custom-libsodium.so")
except OSError as e:
    print(e)
ctypes.cdll.LoadLibrary("/home/sp/test/custom-libsodium.so")

## Using shared library functions

In [ ]:
libc.random()

In [ ]:
libc.printf(b"Hello, %s\n", b"World!")

automatic type conversion is only performed for int, strings and bytes objects

In [ ]:
libc.printf(b"7 // 3 == %d \n", 7 // 3)

# explict type required here
libc.printf(b"but 7 / 3 == %.2f \n", c_double(7 / 3))  

`None` is also used as null pointer

In [ ]:
import time
print(libc.time(None))
print(time.time())

## time(2) - Linux man page
### Name
time - get time in seconds

### Synopsis
```
#include <time.h>  
time_t time(time_t *t);
```

### Description
time() returns the time as the number of seconds since the Epoch, 1970-01-01 00:00:00 +0000 (UTC).
If t is non-NULL, the return value is also stored in the memory pointed to by t.

## Using shared library functions cont.

In [ ]:
# the following line would most likely crash your python interpreter
# libc.time(4711)

In [ ]:
v1, v2 = c_uint32(), c_uint32()

ptr = POINTER(c_uint32)(v1)
r1 = libc.time(ptr)
r2 = libc.time(byref(v2))

r1, r2, v1, v2

In [ ]:
r1 == v1, r2 == v2

## Python callbacks in C Code

In [ ]:
from ctypes import CFUNCTYPE

values = (c_int * 5)(3, 3, 2, 4, 1)

@CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))
def compare_ints(a_ptr, b_ptr):
    a, b = a_ptr[0], b_ptr[0]
    print(f"comparing {a} to {a}")
    return a - b

libc.qsort(values, len(values), sizeof(c_int), compare_ints)

print()
print(" <= ".join(str(v) for v in values))

## A real world example: digital signatures with Libsodium

### Generating a new keypair

In [ ]:
# allocate space for keypair
sk = bytearray(64)
pk = bytearray(32)

# create corresponding ctypes
c_sk = ctypes.ARRAY(c_char, len(sk)).from_buffer(sk)
c_pk = ctypes.ARRAY(c_char, len(pk)).from_buffer(pk)

# randomly generate a fresh keypair
libsodium.crypto_sign_keypair(c_pk, c_sk)

print("sk:", sk[:32].hex())
print("pk:", pk.hex())

### Signing a message

In [ ]:
msg = b"Hello libsodium!"
c_msg = ctypes.create_string_buffer(msg)

sig = bytearray(64)
c_sig = ctypes.ARRAY(c_char, len(sig)).from_buffer(sig)
c_siglen = ctypes.c_longlong()

libsodium.crypto_sign_detached(
    c_sig, byref(c_siglen), c_msg, len(msg), c_sk
) == 0

print(f"sig: {sig[:32].hex()}...")
print("len:", c_siglen.value)

### Verifying a signature

In [ ]:
libsodium.crypto_sign_verify_detached(c_sig, c_msg, len(msg), c_pk) == 0

In [ ]:
c_other_msg = ctypes.create_string_buffer(b"Hello libsodium?")
libsodium.crypto_sign_verify_detached(
    c_sig, c_other_msg, len(c_other_msg), c_pk
) == 0

In [ ]:
invalid_sig = sig[:]
invalid_sig[17] += 1
c_invalid_sig = ctypes.ARRAY(c_char, len(invalid_sig)).from_buffer(invalid_sig)

libsodium.crypto_sign_verify_detached(
    c_invalid_sig, c_other_msg, len(c_msg), c_pk
) == 0

## Writing a sharing library

In [ ]:
!cat foo.h

In [ ]:
!cat foo.c

## Writing a sharing library

In [ ]:
!rm -f foo.o libfoo.so && ls

In [ ]:
!gcc -c -Wall -Werror -fpic foo.c

In [ ]:
!gcc -shared -o libfoo.so foo.o

In [ ]:
!strip -s libfoo.so

In [ ]:
!ls

In [ ]:
libfoo = cdll.LoadLibrary('./libfoo.so')
libfoo.the_answer_to_life_the_universe_and_everything()

Note: Reloading of modified library requires interpreter restart

In [ ]:
with open('/proc/self/maps') as f:
    print(f.read())

In [ ]:
!nm libfoo.so

In [ ]:
!objdump -T libfoo.so

In [ ]:
vars(libfoo)

## A few key takeways

- Profile first
- Check for existing wrappers before writing your own
- Isolate usage
- Write unit tests for your wrappers
- Be careful, but not afraid

# Intro to Python ctypes
### Philipp Schindler

Slides available at:  
https://github.com/PhilippSchindler/ctypes-demo

See also:  
https://docs.python.org/3/library/ctypes.html